## Variational Quantum Classifier

This notebook implements a MSE-based Variational Quantum Classifier to predict bits encrypted through Learning With Errors (a component of Kyber). The model uses Stochastic Gradient Descent.

### 🧠Table of Contents:
 0. 🧮[Data Generation](data-gen/pqc-math-sim.cpp) <br/>
    *Publicly acessible LWE data and encoded bit algebraic simulator*

 1. 🛠️[Initialization and Preprocessing](#initialization-and-preprocessing) <br/>
    *Train-test split and other features*

 2. ⚛️[Define our Quantum Ansatz](#define-our-quantum-simulation) <br/>
    *The circuit employs a Basic and Strong Entangler layer*

 3. 📉[Cost, Accuracy, and Confusion](#cost-accuracy-and-confusion) <br/>
    *Uses Mean Squared Error, with suboptimal hybrid/full BCE options*

 4. 📊[Graphing Functionality](#graphing-functionality) <br/>
    *Uses PlotLy to generate cost and accuracy graphs*

 5. 🔁[Stochastic Gradient Descent](#stochastic-gradient-descent) <br/>
    *Implementation of random-sample SGD with weight initializations*

 6. 🗃️[(Deprecated) Extra Backups](#deprecated-extra-backups) <br/>
    *Backing up this notebook and the cost files*

 7. 🏔️[Training model on new terrain](#training-model-on-new-terrain) <br/>
    *Varying RNG, keys, and increased test sizes reveal decreased generalizability.*

 8. 🖋️[Changelog](#changelog) <br/>
    *My insights and interpretations, as well as documented daily progress.*

### Initialization and Preprocessing

We start by loading the neccesary libraries and variables for the program to function smoothly.

In [1]:
from pennylane import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime  import datetime
import cloudpickle as cp
import pennylane as qml
import pandas as pd
import random
import copy
import os
df = pd.read_csv('data-gen/default.csv')
pi = np.pi

Variational Quantum Classifiers rely on randomness for a variety of things. Let's make sure the randomness isn't too random, so that re-testing and other things are more stable.

In [2]:
random.seed(42)
np.random.seed(42)

Just like classical ML models (and our XGBoost model), we'll make the data into a train-test split.

In [3]:
n_rows = 8000
X_train = df.iloc[0:n_rows, 0:5].to_numpy()
X_test  = df.iloc[n_rows:, 0:5]
Y_train = df.iloc[0:n_rows, 5].to_numpy()   # Labels the model'll run on
Y_test  = df.iloc[n_rows:, 5]               # Labels the model'll check with

In [4]:
print(type(Y_train), X_train.shape)


<class 'numpy.ndarray'> (8000, 5)


These variables wil define the scale of normalization as such
$$
\theta = \text{scale}(x) \in [-\pi, \pi], \quad \text{for } x \in [a, b]
$$

In [5]:
coeff_max = df.iloc[:, 0:4].max().max()
coeff_min = df.iloc[:, 0:4].min().min()
rhs_max   = df.iloc[:, 4].max()
rhs_min   = df.iloc[:, 4].min()

Before encoding the data, let's load the data produced by `pqc-math-sim.cpp` as follows:

In [6]:
def generate_features_and_labels(index):
    features = X_train[index]
    labels = Y_train[index]
    return features, labels

### Define our Quantum Simulation

Since we have 5 features in the CSV data, we can define a 5-qubit system (no data's loaded into it yet):

$$
\left| \psi_0 \right\rangle = \left|0\right\rangle^{\otimes 5} = \left|0\right\rangle \otimes \left|0\right\rangle \otimes \left|0\right\rangle \otimes \left|0\right\rangle \otimes \left|0\right\rangle
$$

In [7]:
dev = qml.device('lightning.qubit', wires=5)

Now let's make some data to translate a `pqc-bits-data.out.csv feature` into $\theta \deg$ that's readable by `pennylane`.

To start, we will have to define a normalization function that translates $|[a, b] \xrightarrow{normalize} [\Delta x, \Delta y]$

In [8]:
def normalize(range_start, range_end, data_point, end_param=pi):
    total_range = range_end - range_start
    shifted = data_point - range_start
    fraction = shifted / total_range
    if end_param == np.pi:
        return (fraction * 2 * end_param) - np.pi
    else:
        return (fraction * end_param)

We define a `circuit`, which uses $R_y(\theta)$ to encode data, where $\theta$ is the numerical feature represented in radians. So like $$|0\rangle \xrightarrow{R_y(\theta)} |\Delta\rangle$$ for each qubit in the multi-qubit system.
 - While defining, we must consider the normalization of each feature into the $\theta \in [0, 2\pi]$ format, as well as the Ansatz, which links things together; here we are using `qml.templates.StronglyEntanglingLayers`
 - The function returns a classification of $\langle Z \rangle:Z ∈ [−1, 1]$ with confidence - so, if the output's closer to $-1$, it's most likely that `actual_bit` is $0$, and vice-versa.


<details>
<summary>Old circuit code</summary>

```python
@qml.qnode(dev)
def old_circuit(parameters):
    for feature in range(len(features)):
        if 0 <= feature < 4:
            qml.RY(normalize(-42, 42, features[feature]), wires=feature)
        elif feature == 4:
            qml.RY(normalize(0, 35, features[feature]), wires=feature)
    qml.templates.StronglyEntanglingLayers(weights=parameters, wires=range(5))
    return qml.expval(qml.Z(0))
```

Note: There was a feature-based dynamic normalization which was removed (because for example, `coeff_1` and `coeff_2` should be normalized along the same interval, but they weren't, which made it harder for the model by adding noise)
</details>



In [9]:
@qml.qnode(dev)
def final_circuit(parameters, features):
    weights_basic, weights_strong = parameters
    for i in range(len(features)):
        if 0 <= i <= 3:
            qml.RY(normalize(coeff_min, coeff_max, features[i]), wires=i)
        if i == 4:
            qml.RY(normalize(rhs_min, rhs_max, features[i]), wires=i)
    qml.templates.StronglyEntanglingLayers(weights=weights_strong, wires=range(5))
    qml.templates.BasicEntanglerLayers(weights=    weights_basic, wires=range(5))
    return qml.expval(qml.Z(0))

### Cost, Accuracy, and Confusion

Let's define a confusion matrix, to show how confused the model is

In [10]:
confusion_matrix = [[0,0],[0,0]]

Note that we swap out the tutorial's `circuit(theta, phi, omega)` for the externally-loaded `circuit(parameters)`

Let's define a cost function. The tutorial uses something, but we'll be using the most common method: **Mean Squared Error**, defined as 
$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2
$$
Which in implementation simplifies to
$$
\text{MSE}_{simple} = |\Delta x - x|^2
$$

Note: `qml.expval(qml.Z(0))` returns a value in $[-1, 1]$, so we normalize it to $[0, 1]$ before computing MSE against classical labels.

Add
<details>
  <summary>Show hidden (old) cost function</summary>

```python
def cost(predicted, actual):
    if predicted == 0:
        nval = 0.5
    elif 1 >= predicted > 0:
        nval = 0.5 + (predicted / 2)
    elif 0 >= predicted >= -1:
        nval = abs(predicted / 2)
    else:
        raise ValueError("Not(-1, 1)")
    
    return abs(nval - actual) ** 2
```
</details>

<details>
  <summary>Show cost function with comments</summary>

```python
def cost(predicted, actual, invert=False):
    if use_mse:
        return abs(normalize(-1, 1, predicted, end_param=1) - actual) ** 2
    # return abs((1 - normalize(-1, 1, predicted, end_param=1)) - actual) ** 2
    # predicted = round(normalize(-1, 1, predicted, end_param=1))
    
    # Get the real prediction, 0 - 1
    real_predicted = normalize(
        -1, 1, predicted, end_param=1
    )

    if invert:
        real_predicted = 1 - real_predicted

    # Confidence is how much it varies from 0. This returns the percent confidence.
    # confidence = abs(0.5 - real_predicted) * 2

    if   actual == 0:
        # Formula: Model 1 - p(y=1)
        confidence = np.log(1 - real_predicted)
    elif actual == 1:
        # Formula: Model p(y=1)
        confidence = np.log(real_predicted)
    return -confidence * (1 / len(X_train))
```
</details>

In [11]:
def cost(predicted, actual, invert=False):
    if use_mse:
        return abs(normalize(-1, 1, predicted, end_param=1) - actual) ** 2

    real_predicted = normalize(
        -1, 1, predicted, end_param=1
    )

    if invert:
        real_predicted = 1 - real_predicted

    if   actual == 0:
        confidence = np.log(1 - real_predicted)
    elif actual == 1:
        confidence = np.log(real_predicted)

    return -confidence * (1 / len(X_train))

<details>
  <summary>Show <code>opt.step</code> flawed code</summary>

Next we set up an optimizer; PennyLane has a number of [built-in optimizers](https://pennylane.readthedocs.io/en/stable/introduction/optimizers.html). This one uses **gradient descent**, defined as:

$$
\frac{\partial f}{\partial \theta} = \frac{f\left(\theta + \frac{\pi}{2}\right) - f\left(\theta - \frac{\pi}{2}\right)}{2}
$$

```python
opt = qml.GradientDescentOptimizer(stepsize=0.1)
```
</details>
<details>
  <summary>Show old cost function(s)</summary>

```python
def cost_fn(weights, X, Y):
    prediction = final_circuit(weights, X)
    return cost(prediction, Y)

def cost_fn(weights, X, Y):
    # If X is 2D (batch), do full loop like before
    if len(X.shape) == 2:
        total_cost = 0
        for i in range(len(X)):
            features = X[i]
            label = Y[i]
            prediction = final_circuit(weights, features)
            total_cost += cost(prediction, label)
        return total_cost / len(X)
    else:
        # Single sample case for SGD
        prediction = final_circuit(weights, X)
        return cost(prediction, Y)
```
</details>

We define a wrapper for `cost(predicted, actual)` where:
 - `weights` are trainable parameters to be updated
 - `X, Y` are features and labels respectively

The wrapper goes over the entire dataset, uses the parameters in `weights` and the given `features` to make a prediction using `final_circuit`. Then the `cost` is calculated, and the total cost over the dataset is returned.

Batch essentially takes the mean over samples of what is normally done, like so:
$$
\texttt{cost\_fn(weights, X, Y)} = \frac{1}{\texttt{len}(X)} \sum_{i=1}^{\texttt{len}(X)} \texttt{cost(final\_circuit(weights, X[i]), Y[i])}
$$

In [12]:
def cost_fn(weights, X, Y):
    if len(X.shape) == 2:
        total_cost = 0
        for i in range(len(X)):
            features = X[i]
            label = Y[i]
            prediction = final_circuit(weights, features)
            total_cost += cost(prediction, label)
        return total_cost / len(X)
    else:
        prediction = final_circuit(weights, X)
        return cost(prediction, Y)


Now, accuracy (it's really just a glorified `final_circuit` wrapper)

In [60]:
def predict(weights, features, reportInteger=False):
        raw_output = final_circuit(weights, features)
        if reportInteger:
            return normalize(-1, 1, raw_output, end_param=1)
        return int(normalize(-1, 1, raw_output, end_param=1) >= 0.5)

In [61]:
def acc_predict_unseen(weights, X_test, Y_test, num_samples=50):
    correct = 0
    total = min(num_samples, len(X_test))
    
    for i in range(total):
        features = X_test.iloc[i].to_numpy()
        label = Y_test.iloc[i]
        prediction = predict(weights, features)
        if prediction == label:
            correct += 1
            
    return correct / total


### Graphing Functionality

Now, let's **graph**...

In [14]:
logpath=""#Will be changed later

Define what we want to graph...

In [15]:
def generate_graph_variables():
    global iters, costs, accs
    iters = [l for l in range(len(graph))]
    costs = [x[1] for x in graph]
    accs = [x[2] for x in graph]

And generate figures...

In [16]:
def generate_plots():
    cost_fig = go.Figure()
    cost_fig.add_trace(go.Scatter(x=iters, y=costs, mode='lines+markers', name='Cost'))
    cost_fig.update_layout(
        title='Training Cost over Iterations',
        xaxis_title='Iteration',
        yaxis_title='Cost',
        template='plotly_dark'
    )

    # Create accuracy plot
    acc_fig = go.Figure()
    acc_fig.add_trace(go.Scatter(x=iters, y=accs, mode='lines+markers', name='Accuracy', line=dict(color='orange')))
    acc_fig.update_layout(
        title='Training Accuracy over Iterations',
        xaxis_title='Iteration',
        yaxis_title='Accuracy',
        template='plotly_dark'
    )

    # Show both
    cost_fig.write_html(logpath + f"\\{current_run}\\cost.html")
    acc_fig.write_html(logpath + f"\\{current_run}\\accuracy.html")


### Stochastic Gradient Descent

Now let's generate weights; a useful reference's the [documentation](https://docs.pennylane.ai/en/stable/code/api/pennylane.StronglyEntanglingLayers.html).

`weights` a.k.a $\theta$ are the trainable parameters that are updated by the program.

<details>
  <summary>Show legacy single-layer or tuple-based weight initialization code</summary>

```python
# Simple
shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=5)
weights = np.random.random(size=shape)
weights = opt.step(lambda w: cost_fn(w, X_train, Y_train), weights)

# Tuple-based
shape_basic = qml.BasicEntanglerLayers.shape(n_layers=2, n_wires=5)
shape_strong = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=5)

weights = (
    np.random.random(size=shape_basic),   # for BasicEntanglerLayers
    np.random.random(size=shape_strong)   # for StronglyEntanglingLayers
)
```
</details>

In [17]:
def reset_weights():
    global weights
    num_qubits = 5
    num_layers_basic = 6
    num_layers_strong = 4

    shape_basic = qml.templates.BasicEntanglerLayers.shape(
        n_layers=num_layers_basic, 
        n_wires=num_qubits
    )

    shape_strong = qml.templates.StronglyEntanglingLayers.shape(
        n_layers=num_layers_strong, 
        n_wires=num_qubits
    )

    weights = (
        np.random.random(size=shape_basic),
        np.random.random(size=shape_strong)
    )

Let's define how often the model should be trained (`n_iter`) and how drastic the changes should be (`stepsize`).

In [18]:
graph = []

<details>
  <summary>Show Normal Gradient Descent Explanation</summary>

*AI-Assisted* This will continually train the model by...
 - Continuously generating `grads`, which is the same shape as (and tells where to nudge) `weights`
 - Applies <u>gradient descent</u>

 $$
 θ ← θ - α ∇θ
 $$
 This is the formula for gradient descent, where
  - $\theta$ = `weights` - a.k.a the trainable parameters
  - $α$ = `stepsize` - a scalar controlling how fast learning is
  - $∇θ$ - the gradient
</details>

*Some AI assistance - understand underlying theory*

Here, we implement <b><a href="https://www.youtube.com/watch?v=qyGPheuLMYc">Stochastic Gradient Descent</a></b>. We must understand that the variable to be updated in these types of algorithms, `weights` $\theta$, are described by an update rule, and that $\theta = [w, b]$. The formula for stochastic gradient descent is like so:
$$
\theta = \theta - \alpha \times dJ/d\theta
$$
Which results into an update on weights, if we substitute $\theta$ for $\theta_0$ a.k.a $w$:
$$
\begin{align*}
w &= w - \alpha \cdot \frac{d}{dw} J \\
  &= w - \alpha \cdot \frac{1}{m} \sum_{i=1}^{m} \left( (w \cdot x_i - b) - y_i \right) \cdot x_i
\end{align*}$$
And a substitution for $b$ a.k.a $\theta$ also reveals the updated bias within $\theta$:
$$
b = b - \alpha \cdot \frac{1}{m} \sum_{i=1}^{m} \left( (w \cdot x_i + b) - y_i \right)
$$

First let's define some initialization parameters...

<sub> AI was used for typesetting LaTeX as well, but not for generating the formulas. </sub>

In [19]:
n_iter = 1600
stepsize = 0.4
use_mse = True

In [20]:
def reset_accuracy_thresholds():
    global best_acc, worst_acc
    best_acc = -1.0       
    worst_acc = 2.0       

In [21]:
def generate_runtime_environment():
    global directory, current_run
    directory = logpath
    current_run = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    os.makedirs(logpath + f"\\{current_run}", exist_ok=True)
    

In [ ]:
logpath = r"backups\night-runtime-6-20-25"

while True:
    reset_weights()
    reset_accuracy_thresholds()
    generate_runtime_environment()
    graph = []
    interrupted = False

    try:

        cost_grad = qml.grad(cost_fn, argnum=0)

        for i in range(n_iter):
            index = np.random.randint(0, len(X_train))
            grads = cost_grad(weights, X_train[index], Y_train[index])
            grads_basic, grads_strong = grads
            # weights = weights - stepsize * np.array(grads)
            weights = (
                weights[0] - stepsize * grads_basic,
                weights[1] - stepsize * grads_strong
            )

            # current_cost = cost_fn(weights, X_train, Y_train)
            current_cost = cost_fn(weights, X_train[index], Y_train[index])
            accuracy = acc_predict_unseen(weights, X_test, Y_test, num_samples=50)
            graph.append([i, current_cost, accuracy, use_mse])
            print(f"Iter {i} | Cost: {current_cost:.5f} | Accuracy: {accuracy:.2f}")

            # if i % 100 == 0:
            #     with open(f"backups/overnight-runtime/{current_run}/checkpoint_iter_{i}.pkl", "wb") as f:
            #         pickle.dump(weights, f)


            if accuracy > best_acc:
                best_acc = accuracy
                with open(logpath + f"\\{current_run}\\best_weights.pkl", "wb") as f:
                    cp.dump(copy.deepcopy(weights), f)
            
            if accuracy < worst_acc:
                worst_acc = accuracy
                with open(logpath + f"\\{current_run}\\worst_weights.pkl", "wb") as f:
                    cp.dump(copy.deepcopy(weights), f)

            # if accuracy >= 0.6 and use_mse:
            #     use_mse = False
            #     print(f"🔁 Switched to BCE")
    except KeyboardInterrupt:
        print("🔁Terminating safely")
        interrupted = True

    with open(logpath + f"\\{current_run}\\graph.pkl", "wb") as f:
        cp.dump(copy.deepcopy(graph), f)
        
    generate_graph_variables()
        
    generate_plots()

    if interrupted:
        break;

### (Deprecated) Extra Backups

Now, let's use `pickle` and `shutil` to back up the current state of the circuit, as well as this notebook itself.

In [ ]:
# with open(directory+"weights_backup_with_inversion_LIVE150_104.pkl", "wb") as f:
#     pickle.dump((weights, len(graph)), f)


In [ ]:
# import pickle, shutil

# directory = "backups/circuit702/"

# with open(directory+"weights_backup.pkl", "wb") as f:
#     pickle.dump((weights, len(graph)), f)

# with open(directory+"training_log.pkl", "wb") as f:
#     pickle.dump(graph, f)

# shutil.copy("accuracy_plot.html", directory+"accuracy.html")
# shutil.copy("cost_plot.html", directory+"cost.html")


### Training model on new terrain

In [27]:
from pennylane import numpy as np
from pathlib import Path
import pandas as pd
import pickle

In [47]:
def all_subpaths(input_path):
    dir_path = Path(input_path)
    paths = list(dir_path.glob("*"))
    return [str(path) for path in paths if "lowkey-trash-runs" not in str(path) and ".cpp" not in str(path)]

In [31]:
def load_data(file_name):
    global df, pi, n_rows, X_train, X_test, Y_train, Y_test
    df = pd.read_csv(file_name)
    pi = np.pi
    df.head()

    n_rows = 8000
    X_train = df.iloc[0:n_rows, 0:5].to_numpy()
    X_test  = df.iloc[n_rows:, 0:5]
    Y_train = df.iloc[0:n_rows, 5].to_numpy()
    Y_test  = df.iloc[n_rows:, 5]

In [62]:
def print_logged_accuracy(path_weight, n_samples=[50], disp=["worst", "best"], terminateEarly=False):
    global best_weights, worst_weights

    if "best" in disp:
        path_best_weights  = path_weight+"\\best_weights.pkl"
        with open(path_best_weights, "rb") as f:
            best_weights = pickle.load(f)

    if "worst" in disp:
        path_worst_weights = path_weight+"\\worst_weights.pkl"
        with open(path_worst_weights, "rb") as f:
            worst_weights = pickle.load(f)

    if terminateEarly:
        return
    
    print(path_weight)
    
    if "worst" in disp:
        for n_sample in n_samples:
            print("Runtime Worst @ test"+str(n_sample)+":", acc_predict_unseen(worst_weights, X_test, Y_test, num_samples=n_sample))
    
    if "best" in disp:
        for n_sample in n_samples:
            print("Runtime Best @ test"+str(n_sample)+":", acc_predict_unseen(best_weights, X_test, Y_test, num_samples=n_sample))


In [59]:
for indiv_path in all_subpaths("data-gen"):
    print(indiv_path)
    load_data(indiv_path)

    print(r"74% run")
    print_logged_accuracy(r"backups\night-runtime-6-20-25\2025-06-20_032842", n_samples=[50, 400], disp=["best", "worst"])

    print(r"24% run")
    print_logged_accuracy(r"backups\night-runtime-6-20-25\2025-06-20_035214", n_samples=[50, 400], disp=["worst", "best"])

data-gen\default.csv
74% run
backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Worst @ test50: 0.66
Runtime Worst @ test400: 0.5575
Runtime Best @ test50: 0.48
Runtime Best @ test400: 0.485
24% run
backups\night-runtime-6-20-25\2025-06-20_035214
Runtime Worst @ test50: 0.5
Runtime Worst @ test400: 0.4875
Runtime Best @ test50: 0.52
Runtime Best @ test400: 0.5475
data-gen\default.mod-key.csv
74% run
backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Worst @ test50: 0.5
Runtime Worst @ test400: 0.5
Runtime Best @ test50: 0.46
Runtime Best @ test400: 0.435
24% run
backups\night-runtime-6-20-25\2025-06-20_035214
Runtime Worst @ test50: 0.54
Runtime Worst @ test400: 0.5425
Runtime Best @ test50: 0.54
Runtime Best @ test400: 0.5025
data-gen\default.mod-rng-and-key.csv
74% run
backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Worst @ test50: 0.56
Runtime Worst @ test400: 0.475
Runtime Best @ test50: 0.64
Runtime Best @ test400: 0.5325
24% run
backups\night-runtime-6-20-25\202

<details>
  <summary>Show archived output</summary>

Evaluation on 50 iters
```
data-gen\default.csv
74% run
backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Worst @ test50: 0.66
Runtime Worst @ test400: 0.5575
Runtime Best @ test50: 0.48
Runtime Best @ test400: 0.485
24% run
backups\night-runtime-6-20-25\2025-06-20_035214
Runtime Worst @ test50: 0.5
Runtime Worst @ test400: 0.4875
Runtime Best @ test50: 0.52
Runtime Best @ test400: 0.5475
data-gen\default.mod-key.csv
74% run
backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Worst @ test50: 0.5
Runtime Worst @ test400: 0.5
Runtime Best @ test50: 0.46
Runtime Best @ test400: 0.435
24% run
backups\night-runtime-6-20-25\2025-06-20_035214
Runtime Worst @ test50: 0.54
Runtime Worst @ test400: 0.5425
Runtime Best @ test50: 0.54
Runtime Best @ test400: 0.5025
data-gen\default.mod-rng-and-key.csv
74% run
backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Worst @ test50: 0.56
Runtime Worst @ test400: 0.475
Runtime Best @ test50: 0.64
Runtime Best @ test400: 0.5325
24% run
backups\night-runtime-6-20-25\2025-06-20_035214
Runtime Worst @ test50: 0.46
Runtime Worst @ test400: 0.51
Runtime Best @ test50: 0.58
Runtime Best @ test400: 0.4975
data-gen\default.mod-rng.csv
74% run
backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Worst @ test50: 0.32
Runtime Worst @ test400: 0.4725
Runtime Best @ test50: 0.74
Runtime Best @ test400: 0.5425
24% run
backups\night-runtime-6-20-25\2025-06-20_035214
Runtime Worst @ test50: 0.24
Runtime Worst @ test400: 0.4975
Runtime Best @ test50: 0.72
Runtime Best @ test400: 0.5175
```
</details>

**Interpretation of the results** The model generalizes differently.
 - Same dataset: Generalizes as expected.
 - Increased iterations: A increased number of random data selections, even if selection earlier with lower iteration count was random, still results in decreased generalizability.
 - Key shift: Model generalizability is gone when the key is changed; it can be inferred that the key forms an integral algebraic and algorithmic component of LWE.
 - RNG and Key shift: Model generalizability retains key aspects as the previously bi-directional model is now gravitating towards one axis of positive accuracy.
   - This suggests that our model is able to capture the complexity of both the same and RNG/key dataset, but missing perhaps an additional layer that could increase generalizability for key shift. However, the implementation of such a layer ~~could screw up the model~~ could threaten both model performance and adaptiveness.
 - RNG Shift: Again, while not as bad as the key shift, some structure is preserved as seen in the worst weights' flipping.

### Graphs for the Research Paper

Let's start by loading in the data for the best and worst weights/$\theta$ values to `best_weights` and `worst_weights`.

In [ ]:
best_run_path  = r"backups\night-runtime-6-20-25\2025-06-20_032842"
worst_run_path = r"backups\night-runtime-6-20-25\2025-06-20_035214"

print_logged_accuracy(best_run_path,  disp=["best"],  terminateEarly=True)
print_logged_accuracy(worst_run_path, disp=["worst"], terminateEarly=True)

Remember: We accidentally trained on `data-gen\default.mod-rng.csv`

In [84]:
df = pd.read_csv('data-gen/default.mod-rng.csv')
pi = np.pi
random.seed(42)
np.random.seed(42)

In [ ]:
from sklearn.metrics import roc_curve, auc

Below code cell tests stuff

In [90]:
print_logged_accuracy(best_run_path,  disp=["best"],  terminateEarly=False)

backups\night-runtime-6-20-25\2025-06-20_032842
Runtime Best @ test50: 0.74


### Terrain 1: Iterative Decay Property

The below code's testing whether the number of iterations of a sample have corellation.

In [92]:
Y_scores = [
    predict(best_weights, x, reportInteger=True)
    for x
    in np.array(X_test, dtype=float)
]

In [122]:
graph_iter_check = []
for n_iters in range(1, 1001, 10):
    _sum = 0
    Y_scores_strict = [round(y) for y in Y_scores]
    Y_test_strict = [x for x in Y_test]
    for i in range(0, n_iters):
        if Y_scores_strict[i] == Y_test_strict[i]:
            _sum += 1
    graph_iter_check.append(_sum / n_iters)

In [123]:
print(len(graph_iter_check))


100


In [124]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, 1001, 10)),
    y=graph_iter_check,
    mode='lines+markers',
    name='Accuracy'
))
fig.update_layout(
    title='VQC Accuracy Over Training Iterations',
    xaxis_title='Training Iterations',
    yaxis_title='Accuracy',
    # yaxis=dict(range=[0, 1]),
    template='plotly_dark'
)
fig.show()


### ROC / AUC

In [85]:
n_rows = 8000
X_train = df.iloc[0:n_rows, 0:5].to_numpy()
X_test  = df.iloc[n_rows:, 0:5]
Y_train = df.iloc[0:n_rows, 5].to_numpy()   # Labels the model'll run on
Y_test  = df.iloc[n_rows:, 5]               # Labels the model'll check with

In [86]:
fpr, tpr, thresholds = roc_curve(Y_test, Y_scores)
roc_auc = auc(fpr, tpr)


In [88]:
Y_scores

[np.float64(0.390796169217535),
 np.float64(0.4380335088396249),
 np.float64(0.3667706395439891),
 np.float64(0.4810104998781413),
 np.float64(0.6057968287724851),
 np.float64(0.46286979243481174),
 np.float64(0.504089366756417),
 np.float64(0.49999868229155753),
 np.float64(0.5106502857678066),
 np.float64(0.5068899876137062),
 np.float64(0.4750178832297926),
 np.float64(0.5027868324862035),
 np.float64(0.455656366406734),
 np.float64(0.5319957428762949),
 np.float64(0.5323547594202299),
 np.float64(0.5295956680738464),
 np.float64(0.6109783791564939),
 np.float64(0.4844631396183432),
 np.float64(0.5725785763978574),
 np.float64(0.36141822462794937),
 np.float64(0.409576838967298),
 np.float64(0.3531475002535557),
 np.float64(0.5829205459205911),
 np.float64(0.5773376033588979),
 np.float64(0.33321556835192123),
 np.float64(0.5622496388840863),
 np.float64(0.5068905587776299),
 np.float64(0.3539217192589446),
 np.float64(0.5704627264793884),
 np.float64(0.5073987236100357),
 np.float6

In [126]:
fig = go.Figure()

# 3. Add ROC curve trace
fig.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'ROC curve (AUC = {roc_auc:.2f})',
    line=dict(color='orange', width=3)
))

# 4. Add diagonal line (random baseline)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random Classifier',
    line=dict(color='gray', dash='dash')
))

# 5. Customize layout
fig.update_layout(
    title='ROC Curve (Quantum Classifier)',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    legend=dict(x=0.65, y=0.05),
    width=700,
    height=500,
    template='plotly_dark'
)

# 6. Show the plot
fig.show()


In [130]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve

# Get ROC data
# fpr, tpr, _ = roc_curve(y_true, y_score)  # Replace with your arrays

# Create y = x baseline
diagonal = fpr  # Since fpr goes from 0 to 1

# Create figure
fig = go.Figure()

# ROC curve
fig.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode='lines',
    name='Model ROC',
    line=dict(color='cyan')
))

# y = x line
fig.add_trace(go.Scatter(
    x=fpr,
    y=diagonal,
    mode='lines',
    name='Random (y = x)',
    line=dict(dash='dash', color='gray')
))

# Shaded: Where ROC is above random
fig.add_trace(go.Scatter(
    x=fpr,
    y=diagonal,
    fill='tonexty',
    mode='none',
    name='Above y = x',
    fillcolor='rgba(0,255,0,0.2)'
))

fig.update_layout(
    title='ROC Curve with Shaded Performance (plotly_dark)',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_dark',
    width=700,
    height=500
)

fig.show()


In [96]:
from sklearn.metrics import roc_auc_score

# y_scores = your model's predicted probabilities for class 1
# Make sure it's a numpy array or list
print("AUC Score:", roc_auc_score(Y_test, Y_scores))


AUC Score: 0.49788151729058694


In [97]:
print("Min score:", np.min(Y_scores))
print("Max score:", np.max(Y_scores))
print("Mean score:", np.mean(Y_scores))


Min score: 0.2443045227584797
Max score: 0.7959745030635179
Mean score: 0.47124858556251936


### Changelog

Past
 - Made QML VQC for cracking PQC, got accuracy 65%

6/18/25
 - Setup overnight runs, BCE failed because my model has low confidence. (Only switch to BCE when optimal weights logged)
 - Cost wobbles at end
 - Accuracy 24% (a.k.a 76% with flipped labels) to 70% in overnight runs; model signaling of peak performance

6/19/25
 - Implemented cloudpickle weight-logging; the normal weight-logging system logged the last weights because of auto-updates (which in our case isn’t a feature because we want to freeze the weights in time)
 - Added CPU optimization
 - **Criticism of paper** Implemented RNG seeds for reproducibility, though the RNG seed variation in real world encryption schemes makes realist implementation difficult, which is why the accuracy is worsened (closer to 50%) when testing on re-generated datasets.
   - There is an argument that an message'll be encrypted using the same dataset and so this may give us an advantage depending on implementation. Real architectures are more likely to use true cryptographic randomness, however.
   - The model should be tested with consistent RNG seeds accross `pqc-math-sim.cpp` and `experiments-2.ipynb` to test whether the model is learning past  quirks of mathematical interactions with the key.
     - However, the model, in real world scenarios, could be used to memorize quirks of 1 key if trained as such.
 - TODO: calcultae bayesian limit, digitize BCE notes and other notes, overnight run again
   - Add a section like "This model is most generealizable when {Implementation of LWE like this}"
 - TO include in paper:
   - Our VQC consistently outperforms classical models such as XGBoost
   - 70% (regular and inverted) = Eval on same dataset with same key
   - 54-inverted% and 60% = Eval with different RNG (partial generalization)
   - 64-inverted% and 62% = Different RNG, Different Key
   - **Also include old sep evaluations that lead to conclusion 2-layered MSE as best**

6/20/25
 - TO Include in paper: 
   - Got 24%(76-inverted) to 74%. QML, especially VQC, is more fragile than classical ML, but outperforms it.
   - The accuracy decrease on increased iterations (past evaluations) isn't because of luck being eliminated. The circuit gets tired or something. Because 50 iters and consistently getting same accuracy on data generated under same conditions still has a point.
   - Preventing circuit getting tired is like future thing. Not my research's focus for now. Maybe one line suggesting `time.sleep()` that sounds fancy (or hardware fatigue?)
 - Know what I need to do to make this a fire paper, don't wanna do it

6/21/25
  - Included ROC/AUC, testing over iterations reveal 80% peak accuracy on 20 samples, with accuracy reducing to 50% on increased samples
  - TODO: Attempt to hack the iteration sample thing to attempt to show 80% accuracy accross entire dataset by splitting it (the circuit gets tired)
  - TODO: Reccomended t-SNE, weight loading, hybrid BCE/MSE based on how strong confidence is to build model confidence
  - Include in paper: 70% accuracy, but ROC/AUC reflect 50% accuracy seen in later iters. Could temporary shoots of accuracy be due to lucky batches?
    - No. We continously test randomly, pulling 50 samples out of 2000 samples.
  - *TL;DR* There is something weird going on